In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
import os
import subprocess as sp
import shutil

# Segmentation dependencies
import sys
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib

import numpy as np
from PIL import Image
import cv2, pdb, glob, argparse

import tensorflow as tf

#importing numba to clean all memory allocated
from numba import cuda

#Matting dependencies
# from __future__ import print_function


import glob, time, argparse, pdb
#import matplotlib.pyplot as plt
import numpy as np
from skimage.measure import label


import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.backends.cudnn as cudnn

from functions import *
from networks import ResnetConditionHR

# Import time to measure time of execution
import time

# Import json to write times
import json
#Tensorflow doesnt release memory after executed(NO used)
import multiprocessing
import re

# %% [markdown]
# ## Separating video in images

# %%
def video2image(input_video, size, input_folder):
    FFMPEG_BIN = "ffmpeg"
    command = [ FFMPEG_BIN,
                '-i', input_video, 
                '-vf', 'scale={}:{}'.format(size[0],size[1]),
                os.path.join(input_folder,"%04d_img.png")]
    print("Extracting frames from video into /input folder...")
    sp.run(command)
    print("Extraction finished successfully")

# %% [markdown]
# ## Running segmentation
# Using deeplabv3 to segment the images

# %%
class DeepLabModel(object):
    """Class to load deeplab model and run inference."""

    INPUT_TENSOR_NAME = 'ImageTensor:0'
    OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
    INPUT_SIZE = 513
    FROZEN_GRAPH_NAME = 'frozen_inference_graph'

    def __init__(self, tarball_path):
        #"""Creates and loads pretrained deeplab model."""
        self.graph = tf.Graph()
        graph_def = None
        # Extract frozen graph from tar archive.
        tar_file = tarfile.open(tarball_path)
        for tar_info in tar_file.getmembers():
            if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
                file_handle = tar_file.extractfile(tar_info)
                graph_def = tf.GraphDef.FromString(file_handle.read())
                break

        tar_file.close()

        if graph_def is None:
            raise RuntimeError('Cannot find inference graph in tar archive.')

        with self.graph.as_default():
            tf.import_graph_def(graph_def, name='')

        #I added this code
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        #----------------
        self.sess = tf.Session(config=config,graph=self.graph)

    def run(self, image):
        """Runs inference on a single image.

        Args:
          image: A PIL.Image object, raw input image.

        Returns:
          resized_image: RGB image resized from original input image.
          seg_map: Segmentation map of `resized_image`.
        """
        width, height = image.size
        resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
        target_size = (int(resize_ratio * width), int(resize_ratio * height))
        resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
        batch_seg_map = self.sess.run(
            self.OUTPUT_TENSOR_NAME,
            feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
        seg_map = batch_seg_map[0]
        return resized_image, seg_map


# %%
def create_pascal_label_colormap():
    """Creates a label colormap used in PASCAL VOC segmentation benchmark.

    Returns:
    A Colormap for visualizing segmentation results.
    """
    colormap = np.zeros((256, 3), dtype=int)
    ind = np.arange(256, dtype=int)

    for shift in reversed(range(8)):
        for channel in range(3):
          colormap[:, channel] |= ((ind >> channel) & 1) << shift
        ind >>= 3

    return colormap

def label_to_color_image(label):
    """Adds color defined by the dataset colormap to the label.

    Args:
    label: A 2D array with integer type, storing the segmentation label.

    Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

    Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
    """
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    colormap = create_pascal_label_colormap()

    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')

    return colormap[label]


In [3]:

## setup ####################

LABEL_NAMES = np.asarray([
        'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
        'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
        'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
])

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)


MODEL_NAME = 'xception_coco_voctrainval'  # @param ['mobilenetv2_coco_voctrainaug', 'mobilenetv2_coco_voctrainval', 'xception_coco_voctrainaug', 'xception_coco_voctrainval']

_DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'
_MODEL_URLS = {
        'mobilenetv2_coco_voctrainaug':
            'deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz',
        'mobilenetv2_coco_voctrainval':
            'deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz',
        'xception_coco_voctrainaug':
            'deeplabv3_pascal_train_aug_2018_01_04.tar.gz',
        'xception_coco_voctrainval':
            'deeplabv3_pascal_trainval_2018_01_04.tar.gz',
}
_TARBALL_NAME = _MODEL_URLS[MODEL_NAME]

model_dir = 'deeplab_model'
if not os.path.exists(model_dir):
    tf.gfile.MakeDirs(model_dir)

download_path = os.path.join(model_dir, _TARBALL_NAME)
if not os.path.exists(download_path):
    print('downloading model to %s, this might take a while...' % download_path)
    urllib.request.urlretrieve(_DOWNLOAD_URL_PREFIX + _MODEL_URLS[MODEL_NAME], 
                        download_path)
    print('download completed! loading DeepLab model...')

MODEL = DeepLabModel(download_path)
print('model loaded successfully!')

#######################################################################################



model loaded successfully!


In [4]:
image = Image.open('/home/josmar/proyectos/codes/annotation_tools/background_substraction/interior1/1280x720/input/0001_img.png')
res_im,seg=MODEL.run(image)
print(seg)
seg=cv2.resize(seg.astype(np.uint8),image.size)
print(seg)
mask_sel=(seg==15).astype(np.float32)
out_mask = (255*mask_sel).astype(np.uint8)


img = cv2.imread('/home/josmar/proyectos/codes/annotation_tools/background_substraction/interior1/1280x720/input/0001_masksDL.png',0)
print(type(out_mask))
print(type(img))




[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [18 18 18 ...  0  0  0]
 [ 0 18 18 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 5 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
kernel = np.ones((5,5),np.uint8)
erosion = cv2.erode(img,kernel,iterations = 1)
closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
opening = cv2.morphologyEx(out_mask, cv2.MORPH_OPEN, kernel)
cv2.imshow('image',opening)
# cv2.imshow('opening',opening)
cv2.waitKey(0)
cv2.destroyAllWindows()

## For videos

In [56]:
def modify_video(folder_path,vid_name, out_folder,kernel,th_low=150,th_up=255):    
    

    complete_path = os.path.join(folder_path,vid_name)

    
    # Python program to save a  
    # video using OpenCV 
    
    
    import cv2 
    
    
    # Create an object to read  
    # from camera 
    video = cv2.VideoCapture(complete_path) 
    
    # We need to check if camera 
    # is opened previously or not 
    if (video.isOpened() == False):  
        print("Error reading video file") 
    
    # We need to set resolutions. 
    # so, convert them from float to integer. 
    frame_width = int(video.get(3)) 
    frame_height = int(video.get(4)) 
    
    size = (frame_width, frame_height) 
    
    # Below VideoWriter object will create 
    # a frame of above defined The output  
    # is stored in 'filename.avi' file.
    new_name = '{}_close.mp4'.format(vid_name.split('.')[0])
    new_name = os.path.join(out_folder,new_name)
    result = cv2.VideoWriter(new_name,  
                            cv2.VideoWriter_fourcc(*'MP4V'), 
                            30, size) 
        
    while(True): 
        ret, frame = video.read() 
    
        if ret == True: 
            _,thresh = cv2.threshold(frame,th_low,th_up,cv2.THRESH_BINARY)

            opening = cv2.morphologyEx(frame, cv2.MORPH_OPEN, kernel)
            close = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
            # Write the frame into the 
            # file 'filename.avi' 
            result.write(close) 
    
            # Display the frame 
            # saved in the file 
            # cv2.imshow('Frame', close) 

            # Press S on keyboard  
            # to stop the process 
            if cv2.waitKey(1) & 0xFF == ord('s'): 
                break
    
        # Break the loop 
        else: 
            break
    
    # When everything done, release  
    # the video capture and video  
    # write objects 
    video.release() 
    result.release() 
        
    # Closes all the frames 
    cv2.destroyAllWindows() 
    
    print("{} was successfully saved".format(new_name))

binary_images/2_720_correction_close.mp4 was successfully saved


In [60]:
input_folder = '/home/josmar/proyectos/Background_Matting/walking_clips/krita_files'
kernel = np.ones((5,5),np.uint8)
th_low = 100
th_up = 255
out_folder = 'bin_close_images'
video_list = os.listdir(input_folder)
video_list.sort()
for v in video_list:
    modify_video(input_folder, v, out_folder, kernel, th_low, th_up)

bin_close_images/10_720_correction_close.mp4 was successfully saved
bin_close_images/12_720_correction_close.mp4 was successfully saved
bin_close_images/13_720_correction_close.mp4 was successfully saved
bin_close_images/14_720_correction_close.mp4 was successfully saved
bin_close_images/15_720_correction_close.mp4 was successfully saved
bin_close_images/16_720_correction_close.mp4 was successfully saved
bin_close_images/18_720_correction_close.mp4 was successfully saved
bin_close_images/19_720_correction_close.mp4 was successfully saved
bin_close_images/20_720_correction_close.mp4 was successfully saved
bin_close_images/22_720_correction_close.mp4 was successfully saved
bin_close_images/23_720_correction_close.mp4 was successfully saved
bin_close_images/24_720_correction_close.mp4 was successfully saved
bin_close_images/25_720_correction_close.mp4 was successfully saved
bin_close_images/26_720_correction_close.mp4 was successfully saved
bin_close_images/27_720_correction_close.mp4 was

In [59]:
input_folder = '/home/josmar/proyectos/Background_Matting/walking_clips/krita_files'
kernel = np.ones((5,5),np.uint8)
th_low = 150
th_up = 255
out_folder = 'more_close'
video_list = os.listdir(input_folder)
video_list.sort()
modify_video(input_folder, "27_720_correction.mp4", out_folder, kernel, th_low, th_up)

more_close/27_720_correction_close.mp4 was successfully saved


In [8]:
print(cv2.countNonZero(img))
print(cv2.countNonZero(opening))

78
0
